# Extraction - Basic
* 고속도로 연결로 접속부 :: 분류부 영향권(2, 3번 영상) 분석

# Imports

In [1]:
import numpy as np
import pandas as pd

import os
from tqdm import tqdm

# Extraction by LOS
* `01_density_by_frame` 폴더의 내용을 바탕으로 LOS별 추출할 프레임을 지정
* **다음을 먼저 작성해야 합니다** : `LOS_frame_list.csv`

## Extracting by LOS
* 각 대상지(영상)별 LOS별로 분석할 프레임만 필터링(20초 이상 = 600개 프레임 이상, LOS별)
* 해당 프레임에 속한 Vehicle ID를 추적
* * `LOS_frame_list`에 명시된 LOS별 프레임들을 추출하여 .csv 파일로 저장

### Load Frame Info

In [2]:
frame_rate = 30 # 프레임율 설정

In [3]:
frm_info_path = 'LOS_frame_list_B.csv'

In [4]:
frm_info = pd.read_csv(frm_info_path)
#frm_info.reset_index(inplace = True)

In [5]:
frm_info['tmsec'] = (frm_info['end_at'] - frm_info['start_at'])/frame_rate # 파생변수 생성: 프레임ID 차이에 따른 시간차

In [6]:
frm_info.head()

,num,LOS,CNT,start_at,end_at,tmsec
0,01_2,B,1,441,3609,105.6
1,01_2,C,1,3609,3849,8.0
2,01_2,B,2,3849,4071,7.4
3,01_2,A,1,4071,4371,10.0
4,01_2,C,2,4494,4836,11.4


### calculate tmsec
* frm_info로부터 각 대상지, LOS별 tmsec 총 관측시간(tmsec) 산정
* 이를 `LOS_tmsec.csv` 로 저장

In [7]:
tmsec_info = pd.pivot_table(frm_info, index = ['num', 'LOS'], values = 'tmsec', aggfunc = 'sum').reset_index()

tmsec_info.head()

,num,LOS,tmsec
0,01_2,A,41.3
1,01_2,B,544.7
2,01_2,C,228.7
3,01_2,D,23.6
4,01_3,A,685.4


In [8]:
tmsec_path = 'LOS_tmsec_B.csv'

In [9]:
tmsec_info.to_csv(tmsec_path, encoding = 'utf-8')

### Load Datasets

In [10]:
folder_dir = 'D:/OneDrive - 연세대학교 (Yonsei University)/Projects/Yonsei_TELab/003_도로상충_210517-/2차년도_2022/16_드론궤적분석자료_2점분합류_금호JC_서울방향/B'

In [11]:
folder_name = '00_dataset_filtered' # 각 .csv 파일이 있는 폴더경로

file_dir = os.path.join(folder_dir, folder_name)

In [12]:
file_list = os.listdir(file_dir)
file_list[0:3]

['01_2.csv', '01_3.csv']

In [13]:
df_list = [] # 데이터프레임 리스트
num_list = [] # 데이터프레임 파일명, 즉 넘버링 리스트

for file in file_list:
    num = file[0:4]
    file_path = os.path.join(file_dir, file)
            
    if os.path.isfile(file_path): # 해당 file_path에 파일이 존재할 경우
        globals()[f'target{num}'] = pd.read_csv(file_path, encoding = 'cp949') # .csv 파일을 불러와 인스턴스 만들기
        df_list.append(globals()[f'target{num}']) 
        num_list.append(num)

In [16]:
target01_2.head(3)

,Unnamed: 0,Unnamed: 0.1,Vehicle ID,Frame ID,Total Frames,Global Time (Epoch Time),Local X (m),Local Y(m),Vehicle Length,Vehicle Width,...,Vehicle Acceleration,Lane Identification,Preceding Vehicle,Following Vehicle,Spacing Headway,Time Headway,Bad Object,Special Car,Lane Class,Vehicle Movement
0,0,0,3,354,39,46811800,203.646042,20.099760,4.936709,1.898734,...,-0.299115,U3,0,0,0.0,0.0,0,0,상행본선,2.426485
1,1,1,4,354,39,46811800,198.960190,24.644833,6.582278,1.898734,...,0.124783,U2,0,0,0.0,0.0,0,0,상행본선,2.705503
2,2,2,3,357,42,46811900,201.215576,20.082336,4.936709,1.898734,...,0.404358,U3,0,0,0.0,0.0,0,0,상행본선,2.430528


### Extract & Save

In [17]:
folder_name = '01_density_by_target_LOS'

save_extract_dir = os.path.join(folder_dir, folder_name)

os.makedirs(save_extract_dir, exist_ok = True) # 해당 경로가 없을 시 폴더 생성, 존재할 경우 건너뛰기

In [18]:
for i in range(len(frm_info)): # frm_info에 따라서
    
    num = frm_info['num'][i] # 대상지 번호. tg_spot. 01_1 등으로 표현
    LOS = frm_info['LOS'][i] # 대상지 촬영지점의 해당 LOS
    CNT = frm_info['CNT'][i] # 몇번쨰 LOS인지 표시
    
    frame_start = frm_info['start_at'][i] # 시작 프레임
    frame_end = frm_info['end_at'][i] # 끝 프레임
    
    tmsec = frm_info['tmsec'][i] # 총 시간
    
    df = globals()[f'target{num}']
    
    # LOS별로 프레임을 추출하여 각 대상지별, LOS별 파일로 저장
    df_extract = df[(df['Frame ID'] >= frame_start) & (df['Frame ID'] <= frame_end)] # 프레임별로 필터링한 df
    
    save_extract_name = f'{num}_{LOS}_{CNT}.csv'
    save_extract_path = os.path.join(save_extract_dir, save_extract_name)
    
    df_extract.to_csv(save_extract_path, encoding = 'cp949')

## Concat and Save
* `01_denstiy_by_target_LOS` 에 있는 .csv 파일들을 대상지별, LOS별로 병합
* `01_denstiy_by_target_LOS_concated`에 저장하기

### Load Dataset
* 대상지 및 LOS 조각별로 쪼개진 데이터들을 불러오기

In [19]:
folder_name = '01_density_by_target_LOS'

file_dir = os.path.join(folder_dir, folder_name)

In [20]:
file_list = os.listdir(file_dir)
file_list[0:3]

['01_2_A_1.csv', '01_2_A_2.csv', '01_2_A_3.csv']

In [21]:
df_list = []
num_list = []
LOS_list = []
fraction_list = []


for file in file_list:
    num = file[0:4] # 파일의 대상지 번호. tg_spot
    LOS = file[5]
    fraction = file[7:-4] # 파일의 조각 번호
    
    file_path = os.path.join(file_dir, file)
    globals()[f'target{num}_{LOS}_{fraction}'] = pd.read_csv(file_path, encoding = 'cp949')
    
    df_list.append(globals()[f'target{num}_{LOS}_{fraction}'])
    
    if num not in num_list:
        num_list.append(num)
    
    if LOS not in LOS_list:
        LOS_list.append(LOS)
        
    if fraction not in fraction_list:
        fraction_list.append(fraction)

In [23]:
target01_2_A_1.head(3)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Vehicle ID,Frame ID,Total Frames,Global Time (Epoch Time),Local X (m),Local Y(m),Vehicle Length,...,Vehicle Acceleration,Lane Identification,Preceding Vehicle,Following Vehicle,Spacing Headway,Time Headway,Bad Object,Special Car,Lane Class,Vehicle Movement
0,11840,12142,12142,329,4071,206,46935700,53.231415,15.266999,5.822785,...,-1.314545,U5,0,35,0.000000,0.000000,0,0,상행본선,2.554237
1,11841,12143,12143,331,4071,201,46935700,62.116364,25.108927,5.443038,...,2.484300,U2,0,37,0.000000,0.000000,0,0,상행본선,2.607176
2,11842,12144,12144,335,4071,151,46935700,107.361649,14.114642,6.329114,...,-2.312533,U5,29,38,54.062637,2.288326,0,0,상행본선,2.362541


### RUN and Save

In [24]:
folder_name = '01_denstiy_by_target_LOS_concated'

save_dir = os.path.join(folder_dir, folder_name)

os.makedirs(save_dir, exist_ok = True) # 해당 경로가 없을 시 폴더 생성, 존재할 경우 건너뛰기

In [25]:
fraction_list

['1',
 '2',
 '3',
 '4',
 '10',
 '11',
 '12',
 '13',
 '14',
 '5',
 '6',
 '7',
 '8',
 '9',
 '15',
 '16',
 '17',
 '18',
 '19']

In [26]:
for num in num_list:
    for LOS in LOS_list:
        LOS_df = pd.DataFrame() # 대상지별, LOS별 빈 데이터프레임 생성
        
        for fraction in fraction_list:
            
            if f'target{num}_{LOS}_{fraction}' in globals(): # 해당 이름의 변수가 전역변수(globals()) 내에 존재할 시
                frac_df = globals()[f'target{num}_{LOS}_{fraction}']
                LOS_df = pd.concat([LOS_df, frac_df])
            else:
                pass
        
        LOS_save_name = num + '_' + LOS + '.csv'
        LOS_save_path = os.path.join(save_dir, LOS_save_name)
        
        LOS_df.to_csv(LOS_save_path, encoding = 'cp949') # LOS별로 병합된 파일을 저장